In [2]:
!pip install fuzzywuzzy python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 24.0 MB/s eta 0:00:00


In [4]:
#preprocess the data

import nltk
nltk.download('punkt')

import nltk
import pandas as pd
import string
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Load anime data from CSV file
df = pd.read_csv("anime.csv")

df[['name','genre']] = df[['name','genre']].fillna('')
print(df.shape)

#punctuation removal 
print(string.punctuation)

#defining the function to remove punctuation
def remove_punctuation(text):
  punctuationfree = "".join([i for i in text if i not in string.punctuation])
  return punctuationfree

#storing the puntuation free text
df['name'] = df['name'].apply(lambda x:remove_punctuation(x))
df['genre'] = df['genre'].apply(lambda x:remove_punctuation(x))

print(df.head)

#lower case 
df['name']= df['name'].apply(lambda x: x.lower())
df['genre']= df['genre'].apply(lambda x: x.lower())

print(df.head)

(12294, 7)
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
<bound method NDFrame.head of        anime_id                                               name  \
0         32281                                      Kimi no Na wa   
1          5114                    Fullmetal Alchemist Brotherhood   
2         28977                                           Gintama°   
3          9253                                         SteinsGate   
4          9969                                         Gintama039   
...         ...                                                ...   
12289      9316         Toushindai My Lover Minami tai MechaMinami   
12290      5543                                        Under World   
12291      5621                     Violence Gekiga David no Hoshi   
12292      6133  Violence Gekiga Shin David no Hoshi Inma Densetsu   
12293     26081                      Yasuji no Pornorama Yacchimae   

                                                   genre   type episodes  \
0        

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
#joint code
#MAIN CODE

#Taking user input as the anime title
anime_input = input("Enter the name of the anime you have watched: ")

#preprocessing the input value 
anime_input = anime_input.lower()
anime_input = remove_punctuation(anime_input)


# Create a TF-IDF matrix for the anime titles
title_vectorizer = TfidfVectorizer()
title_tfidf_matrix = title_vectorizer.fit_transform(df['name'])

# Define a function to calculate the cosine similarity between the input query and the anime titles
def get_title_similarity(query):
    query_tfidf = title_vectorizer.transform([query])
    cosine_similarities = cosine_similarity(query_tfidf, title_tfidf_matrix)[0]
    return cosine_similarities

# Define a function to find the closest matching anime title using FuzzyWuzzy
def get_closest_title(query):
    closest_title = None
    closest_title_score = -1
    
    for title in df['name']:
        score = fuzz.token_sort_ratio(query, title)
        if score > closest_title_score:
            closest_title = title
            closest_title_score = score
    
    print('FUZYWUZZZY',closest_title, closest_title_score)
    return closest_title, closest_title_score

# Define a function to recommend similar anime titles based on the input query
def recommend_anime(query):
    # Find the most similar anime titles using both TF-IDF and FuzzyWuzzy
    title_similarities = get_title_similarity(query) #cosine
    #numeric data
    title_scores = [(df.iloc[i]['name'], score) for i, score in enumerate(title_similarities)]
    print(title_scores)
    closest_title, closest_title_score = get_closest_title(query)
    title_scores.append((closest_title, closest_title_score))
    print(title_scores)
    title_scores = sorted(title_scores, key=lambda x: x[1], reverse=True)[:5]
    print(title_scores)
    
    # Recommend similar anime titles based on the most similar anime titles' genres
    genres = set()
    for title, _ in title_scores:
        title_genres = df[df['name'] == title]['genre'].values[0].split(',')
        genres.update(title_genres)
        
    similar_df = df[df['genre'].str.contains('|'.join(genres))]
    print(similar_df)
    similar_df = similar_df[~similar_df['name'].isin([title for title, _ in title_scores])]
    print(similar_df)
    similar_df = similar_df.sort_values(by='members', ascending=False).head(5)
    print(similar_df)
    return similar_df['name'].tolist()

# Example usage:
query = anime_input
recommended_anime = recommend_anime(query)
print()

get_closest_title(query)

print(f"Recommended anime titles similar to '{query}':")
print()

for title in recommended_anime:
    print(title.upper())

Enter the name of the anime you have watched: naruto
[('kimi no na wa', 0.0), ('fullmetal alchemist brotherhood', 0.0), ('gintama°', 0.0), ('steinsgate', 0.0), ('gintama039', 0.0), ('haikyuu karasuno koukou vs shiratorizawa gakuen koukou', 0.0), ('hunter x hunter 2011', 0.0), ('ginga eiyuu densetsu', 0.0), ('gintama movie kanketsuhen  yorozuya yo eien nare', 0.0), ('gintama039 enchousen', 0.0), ('clannad after story', 0.0), ('koe no katachi', 0.0), ('gintama', 0.0), ('code geass hangyaku no lelouch r2', 0.0), ('haikyuu second season', 0.0), ('sen to chihiro no kamikakushi', 0.0), ('shigatsu wa kimi no uso', 0.0), ('mushishi zoku shou 2nd season', 0.0), ('ookami kodomo no ame to yuki', 0.0), ('code geass hangyaku no lelouch', 0.0), ('hajime no ippo', 0.0), ('rurouni kenshin meiji kenkaku romantan  tsuiokuhen', 0.0), ('cowboy bebop', 0.0), ('one punch man', 0.0), ('mononoke hime', 0.0), ('suzumiya haruhi no shoushitsu', 0.0), ('monogatari series second season', 0.0), ('mushishi zoku shou

In [ ]:
#FuzzyWuzzy: Only

# Calculate similarity scores between the user input and all anime titles
scores = df['name'].apply(lambda x: fuzz.token_set_ratio(x, anime_input))

# Sort the dataframe by similarity scores
sorted_df = df.loc[scores.sort_values(ascending=False).index]

# Print the top 5 anime recommendations
top_recommendations = sorted_df.head()['name'].tolist()
print(f'Top 5 anime recommendations for "{anime_input}":')
print()
for i, recommendation in enumerate(top_recommendations):
    print(f'{i+1}. {recommendation}')

Top 5 anime recommendations for "naruto":

1. naruto the cross roads
2. the last naruto the movie
3. naruto movie 2 dai gekitotsu maboroshi no chiteiiseki dattebayo
4. naruto shippuuden movie 3  hi no ishi wo tsugu mono
5. naruto movie 3 dai koufun mikazuki jima no animaru panikku dattebayo


In [ ]:
#TFIDF Vectorizer: Only

# Preprocess the data
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

preprocessed_data = []
for summary in df['name']:
    #word_tokenize converts words into seperate tokens
    words = nltk.word_tokenize(summary)

    words = [word for word in words if word.isalpha() and word not in stopwords]
    print(words)
    preprocessed_data.append(" ".join(words))


df['preprocessed_summary'] = preprocessed_data

# Vectorize the data
vectorizer = TfidfVectorizer()
vectorized_data = vectorizer.fit_transform(df['preprocessed_summary'])

# Calculate similarity scores
similarity_matrix = cosine_similarity(vectorized_data)

# Implement the recommendation engine
print(df['name'].head)

def recommend_anime(title):
    index = df.index[df['name'] == title].tolist()[0]
    scores = list(enumerate(similarity_matrix[index]))
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    recommended_anime = [df.iloc[score[0]]['name'] for score in sorted_scores[1:4]]
    return recommended_anime

print(recommend_anime(anime_input))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


AttributeError: ignored